<a href="https://colab.research.google.com/github/da-head0/cctv-anomaly-detection/blob/main/I3D_ConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv3D, MaxPooling3D, Flatten, Dropout

- 영상 높이를 128로 하는 이유는 이 사이즈가 가장 파라미터 수 대비 계산 효율과 성능이 좋게 나오기 때문이다.

In [5]:
# 아예 이 모델을 3D로 다 바꿔야 하나?
base_model = tf.keras.applications.InceptionV3(input_shape = (256,256,3), # 3이 맨 앞으로 가도 됨
                                               include_top=False, # include fully connected layer at the top
                                               weights='imagenet') # 224,224로 크롭? - 150,150, 이상이기만 하면 됨. include top 할거면 229,229
base_model.trainable = False # free model to prevents the weights not updated
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 640, 360, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 319, 179, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 319, 179, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 319, 179, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

- 256x256으로 하면 파라미터 218만
- 640x360으로 줄여도 이 파라미터 개수는 똑같음.

In [ ]:
inputs = tf.keras.Input(shape=(256, 258, 3))
#x = data_augmentation(inputs) # 증강
#x = preprocess_input(x) # 전처리 함수
x = base_model(x, training=False)
# fully connected layer?
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# last averaging pooling layer
# lstm with batch mormalization

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
def get_model():
  model = Sequential()
  input_shape = (16, 128, 171, 3) # 프레임 개수, 영상 높이, 영상 너비, 채널 수

  # 1st layer     # filter, kernal-높이,깊이,너비의 크기 - 3D 요소를 형성. 3,3,3이 가장 성능이 좋더라...
  # padding = same으로 하면 패딩이 균등하게 이루어진다. 
  model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv1', input_shape=input_shape))
  
  # stride = 1로 input output size 차이가 없게 함
  # pooling은 첫번째 레이어만 1x2x2 - 시간적 신호를 너무 일찍 merge 시키지 않기 위해서
  model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,1,1), padding='valid', name='pool1'))

  # 2nd layer
  model.add(Conv3D(128,kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv2', input_shape=input_shape))
  model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1), padding='valid', name='pool2'))

  # 3rd layer
  model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv3', input_shape=input_shape))
  model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1), padding='valid', name='pool3'))

  # 4th layer
  model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv4', input_shape=input_shape))
  model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1), padding='valid', name='pool4'))

  # 5th layer
  model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu',padding='same', name='conv5', input_shape=input_shape))
  model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1), padding='valid', name='pool5'))
  
  model.add(Flatten())# fully connected로 처리하려면 넣어주어야 하나...

# 2 fully connected layers - 2048 outputs
  model.add(Dense(2048, activation='relu', name='fc1'))
  model.add(Dropout(.5))

  model.add(Dense(2048, activation='relu', name='fc2'))
  model.add(Dropout(.5))

  # softmax loss layer
  model.add(Dense(20, activation='softmax'))
  model.summary()

get_model()

In [ ]:
# Model configuration
batch_size = 30
no_epochs = 16
learning_rate = 0.003 # divided by 10 after every 4 epochs
no_classes = 13 # 분류할 행동의 갯수